In [1]:
import pathlib
import tempfile
from urllib.parse import urlparse
from urllib.request import urlretrieve

import numpy as np
import asdf
from astropy.io import fits
from astropy.modeling import models, fitting
import astropy.units as u

from gwcs import wcs as gwcs
from gwcs import coordinate_frames as cf

from HST.hst_grism_reffiles import create_tsgrism_wavelengthrange, create_grism_specwcs
from HST.generate_wfc3_distortion import create_wfc3_distortion

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-034 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-036
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf


In [ ]:
reference_files = dict()

conf_filepath = "HST/UVIS_G280_CCD1_V2.conf"

filter = "G280"

specwcs_filename = "WFC3_" + str(filter) + "_specwcs.asdf"
create_grism_specwcs(conffile=str(conf_filepath), pupil=filter, outname=specwcs_filename)

#### Create inverse SIP coefficients for UVIS

In [2]:
sip_file = "/Users/rosteen/Data/UVIS/G280/icwz15e7q_flt.fits"
sip_hdus = fits.open(str(sip_file))

# CCD 1
acoef_1 = dict(sip_hdus[1].header['A_*'])
a_order_1 = acoef_1.pop('A_ORDER')
bcoef_1 = dict(sip_hdus[1].header['B_*'])
b_order_1 = bcoef_1.pop('B_ORDER')

# CCD 2
acoef_2 = dict(sip_hdus[4].header['A_*'])
a_order_2 = acoef_2.pop('A_ORDER')
bcoef_2 = dict(sip_hdus[4].header['B_*'])
b_order_2 = bcoef_2.pop('B_ORDER')

In [3]:
acoef_1

{'A_0_2': 9.46198776524919e-08,
 'A_0_3': 1.90387041960406e-11,
 'A_0_4': -1.6373003756199e-14,
 'A_1_1': -2.9709966754903e-06,
 'A_1_2': 1.85317727498633e-11,
 'A_1_3': 3.09103330524291e-15,
 'A_2_0': 2.87424293769154e-06,
 'A_2_1': -1.5590800935127e-11,
 'A_2_2': -1.7808492046889e-14,
 'A_3_0': 2.04158611679013e-11,
 'A_3_1': -8.3888624793725e-16,
 'A_4_0': 5.34875121096486e-16}

In [4]:
acoef_2

{'A_0_2': -1.064800921188e-08,
 'A_0_3': 2.36026897013524e-12,
 'A_0_4': 5.07730159533929e-15,
 'A_1_1': -2.9211535091318e-06,
 'A_1_2': 1.99343356293459e-11,
 'A_1_3': 1.59872109528426e-14,
 'A_2_0': 2.84447497246753e-06,
 'A_2_1': 5.29959356829531e-12,
 'A_2_2': -1.2690082837373e-16,
 'A_3_0': 7.74851553027805e-12,
 'A_3_1': 7.6935679249928e-15,
 'A_4_0': 2.97589767803805e-15}

In [5]:
sip_hdus[1].data.shape

(2051, 4096)

In [6]:
indices = np.indices((2051, 4096))
print(indices)

[[[   0    0    0 ...    0    0    0]
  [   1    1    1 ...    1    1    1]
  [   2    2    2 ...    2    2    2]
  ...
  [2048 2048 2048 ... 2048 2048 2048]
  [2049 2049 2049 ... 2049 2049 2049]
  [2050 2050 2050 ... 2050 2050 2050]]

 [[   0    1    2 ... 4093 4094 4095]
  [   0    1    2 ... 4093 4094 4095]
  [   0    1    2 ... 4093 4094 4095]
  ...
  [   0    1    2 ... 4093 4094 4095]
  [   0    1    2 ... 4093 4094 4095]
  [   0    1    2 ... 4093 4094 4095]]]


In [8]:
indices.shape

(2, 2051, 4096)

In [9]:
x_indices = indices[0,:]
y_indices = indices[1,:]

In [10]:
a_polycoef_1 = {}
for key in acoef_1:
    a_polycoef_1['c' + key.split('A_')[1]] = acoef_1[key]

b_polycoef_1 = {}
for key in bcoef_1:
    b_polycoef_1['c' + key.split('B_')[1]] = bcoef_1[key]
    
a_polycoef_2 = {}
for key in acoef_2:
    a_polycoef_2['c' + key.split('A_')[1]] = acoef_2[key]

b_polycoef_2 = {}
for key in bcoef_2:
    b_polycoef_2['c' + key.split('B_')[1]] = bcoef_2[key]

In [11]:
a_poly_1 = models.Polynomial2D(a_order_1, **a_polycoef_1)
b_poly_1 = models.Polynomial2D(b_order_1, **b_polycoef_1)

a_poly_2 = models.Polynomial2D(a_order_2, **a_polycoef_2)
b_poly_2 = models.Polynomial2D(b_order_2, **b_polycoef_2)

In [12]:
result_x_1 = a_poly_1(x_indices, y_indices)
result_y_1 = b_poly_1(x_indices, y_indices)

result_x_2 = a_poly_2(x_indices, y_indices)
result_y_2 = b_poly_2(x_indices, y_indices)

In [13]:
result_x_1

array([[ 0.00000000e+00,  9.46389000e-08,  3.78631558e-07, ...,
        -1.70451320e+00, -1.70727381e+00, -1.71003705e+00],
       [ 2.87426335e-06, -2.09149600e-09, -2.68905554e-06, ...,
        -1.71614857e+00, -1.71891185e+00, -1.72167775e+00],
       [ 1.14971351e-05,  5.64975526e-06, -8.19677860e-09, ...,
        -1.72777892e+00, -1.73054486e+00, -1.73331343e+00],
       ...,
       [ 1.22402292e+01,  1.22340720e+01,  1.22279150e+01, ...,
        -1.48471451e+01, -1.48560455e+01, -1.48649484e+01],
       [ 1.22522804e+01,  1.22461202e+01,  1.22399601e+01, ...,
        -1.48482588e+01, -1.48571625e+01, -1.48660688e+01],
       [ 1.22643376e+01,  1.22581743e+01,  1.22520112e+01, ...,
        -1.48493672e+01, -1.48582743e+01, -1.48671839e+01]])

In [14]:
x_indices + result_x_1

array([[ 0.00000000e+00,  9.46389000e-08,  3.78631558e-07, ...,
        -1.70451320e+00, -1.70727381e+00, -1.71003705e+00],
       [ 1.00000287e+00,  9.99999998e-01,  9.99997311e-01, ...,
        -7.16148574e-01, -7.18911846e-01, -7.21677755e-01],
       [ 2.00001150e+00,  2.00000565e+00,  1.99999999e+00, ...,
         2.72221077e-01,  2.69455141e-01,  2.66686568e-01],
       ...,
       [ 2.06024023e+03,  2.06023407e+03,  2.06022792e+03, ...,
         2.03315285e+03,  2.03314395e+03,  2.03313505e+03],
       [ 2.06125228e+03,  2.06124612e+03,  2.06123996e+03, ...,
         2.03415174e+03,  2.03414284e+03,  2.03413393e+03],
       [ 2.06226434e+03,  2.06225817e+03,  2.06225201e+03, ...,
         2.03515063e+03,  2.03514173e+03,  2.03513282e+03]])

In [15]:
# Initialize a 5th degree 2D polynomial for fitting
inverse_model = models.Polynomial2D(5)

In [16]:
inverse_model

<Polynomial2D(5, c0_0=0., c1_0=0., c2_0=0., c3_0=0., c4_0=0., c5_0=0., c0_1=0., c0_2=0., c0_3=0., c0_4=0., c0_5=0., c1_1=0., c1_2=0., c1_3=0., c1_4=0., c2_1=0., c2_2=0., c2_3=0., c3_1=0., c3_2=0., c4_1=0.)>

In [17]:
fitter = fitting.LevMarLSQFitter()

best_fit = fitter(inverse_model, x_indices+result_x_1, y_indices+result_y_1, -result_x_1)
x_inverse_model = best_fit

best_fit = fitter(inverse_model, x_indices+result_x_1, y_indices+result_y_1, -result_y_1)
y_inverse_model = best_fit

In [18]:
x_inverse_model

<Polynomial2D(5, c0_0=0.00003937, c1_0=-0.00000007, c2_0=-0.00000287, c3_0=-0., c4_0=-0., c5_0=0., c0_1=-0.00000029, c0_2=-0.00000009, c0_3=-0., c0_4=0., c0_5=0., c1_1=0.00000297, c1_2=0., c1_3=-0., c1_4=-0., c2_1=-0., c2_2=0., c2_3=0., c3_1=0., c3_2=-0., c4_1=-0.)>

In [19]:
y_inverse_model

<Polynomial2D(5, c0_0=-0.00000043, c1_0=0.00000002, c2_0=0.00000004, c3_0=-0., c4_0=-0., c5_0=0., c0_1=-0.00000001, c0_2=0.00000308, c0_3=-0., c0_4=-0., c0_5=-0., c1_1=-0.00000282, c1_2=-0., c1_3=0., c1_4=0., c2_1=-0., c2_2=-0., c2_3=-0., c3_1=-0., c3_2=0., c4_1=-0.)>

In [20]:
x_inverse_model(x_indices+result_x_1, y_indices+result_y_1)

array([[ 3.93653010e-05,  3.89786389e-05,  3.84037960e-05, ...,
         1.70458127e+00,  1.70734243e+00,  1.71010623e+00],
       [ 3.64231529e-05,  3.90077474e-05,  4.14041235e-05, ...,
         1.71621653e+00,  1.71898036e+00,  1.72174683e+00],
       [ 2.77325104e-05,  3.32883923e-05,  3.86560184e-05, ...,
         1.72784678e+00,  1.73061327e+00,  1.73338240e+00],
       ...,
       [-1.22401934e+01, -1.22340365e+01, -1.22278798e+01, ...,
         1.48471893e+01,  1.48560900e+01,  1.48649933e+01],
       [-1.22522446e+01, -1.22460846e+01, -1.22399248e+01, ...,
         1.48483030e+01,  1.48572071e+01,  1.48661137e+01],
       [-1.22643017e+01, -1.22581388e+01, -1.22519759e+01, ...,
         1.48494115e+01,  1.48583189e+01,  1.48672289e+01]])

In [21]:
(x_inverse_model(x_indices+result_x_1, y_indices+result_y_1) + result_x_1).max()

6.917971317910165e-05

In [22]:
(y_inverse_model(x_indices+result_x_1, y_indices+result_y_1) + result_y_1).max()

1.1541708708762144e-05

In [ ]:
fitter = fitting.LevMarLSQFitter()

best_fit = fitter(inverse_model, x_indices+result_x_2, y_indices+result_y_2, -result_x_2)
x_inverse_model_2 = best_fit

best_fit = fitter(inverse_model, x_indices+result_x_2, y_indices+result_y_2, -result_y_2)
y_inverse_model_2 = best_fit

In [ ]:
# Testing to see if this actually worked...

crpix = [sip_hdus[1].header['CRPIX1'], sip_hdus[1].header['CRPIX2']]

crval = [sip_hdus[1].header['CRVAL1'],
         sip_hdus[1].header['CRVAL2']]

cdmat = np.array([[sip_hdus[1].header['CD1_1'], sip_hdus[1].header['CD1_2']],
                  [sip_hdus[1].header['CD2_1'], sip_hdus[1].header['CD2_2']]])

SIP_forward = (models.Shift(-(crpix[0]-1)) & models.Shift(-(crpix[1]-1)) | # Calculate u and v
              models.Mapping((0, 1, 0, 1, 0, 1)) | a_poly_1 & b_poly_1 & models.Identity(2) |
              models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
              models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() |
              models.RotateNative2Celestial(crval[0], crval[1], 180))

SIP_backward = (models.RotateCelestial2Native(crval[0], crval[1], 180) |
             models.Sky2Pix_TAN() | models.AffineTransformation2D(matrix=cdmat).inverse |
             models.Mapping((0, 1, 0, 1, 0, 1)) | x_inverse_model & y_inverse_model & models.Identity(2) |
             models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
             models.Shift((crpix[0]-1)) & models.Shift((crpix[1]-1)))

full_distortion_model = SIP_forward
full_distortion_model.inverse = SIP_backward

imagepipe = []

det_frame = cf.Frame2D(name="detector")
imagepipe.append((det_frame, full_distortion_model))
world_frame = cf.CelestialFrame(name="world", unit = (u.Unit("deg"), u.Unit("deg")),
                                axes_names=('lon', 'lat'), axes_order=(0, 1),
                                reference_frame="ICRS")
imagepipe.append((world_frame, None))
geo_transforms = gwcs.WCS(imagepipe)

In [ ]:
geo_transforms.available_frames

In [ ]:
forward = geo_transforms.get_transform("detector", "world")

In [ ]:
backward = geo_transforms.get_transform("world", "detector")

In [ ]:
forward(2048, 1026)

In [ ]:
backward(206.43128029664106, 26.4186090137141)

In [ ]:
# Now test the models for CCD2

crpix = [sip_hdus[4].header['CRPIX1'], sip_hdus[4].header['CRPIX2']]

crval = [sip_hdus[4].header['CRVAL1'],
         sip_hdus[4].header['CRVAL2']]

cdmat = np.array([[sip_hdus[4].header['CD1_1'], sip_hdus[4].header['CD1_2']],
                  [sip_hdus[4].header['CD2_1'], sip_hdus[4].header['CD2_2']]])

SIP_forward = (models.Shift(-(crpix[0]-1)) & models.Shift(-(crpix[1]-1)) | # Calculate u and v
              models.Mapping((0, 1, 0, 1, 0, 1)) | a_poly_2 & b_poly_2 & models.Identity(2) |
              models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
              models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() |
              models.RotateNative2Celestial(crval[0], crval[1], 180))

SIP_backward = (models.RotateCelestial2Native(crval[0], crval[1], 180) |
             models.Sky2Pix_TAN() | models.AffineTransformation2D(matrix=cdmat).inverse |
             models.Mapping((0, 1, 0, 1, 0, 1)) | x_inverse_model_2 & y_inverse_model_2 & models.Identity(2) |
             models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
             models.Shift((crpix[0]-1)) & models.Shift((crpix[1]-1)))

full_distortion_model = SIP_forward
full_distortion_model.inverse = SIP_backward

imagepipe = []

det_frame = cf.Frame2D(name="detector")
imagepipe.append((det_frame, full_distortion_model))
world_frame = cf.CelestialFrame(name="world", unit = (u.Unit("deg"), u.Unit("deg")),
                                axes_names=('lon', 'lat'), axes_order=(0, 1),
                                reference_frame="ICRS")
imagepipe.append((world_frame, None))
geo_transforms = gwcs.WCS(imagepipe)

In [ ]:
forward = geo_transforms.get_transform("detector", "world")
backward = geo_transforms.get_transform("world", "detector")

In [ ]:
forward(2048, 1026)

In [ ]:
backward(206.4568554451976, 26.418285311669543)

In [ ]:
x_inverse_model_2